# Advection Equation (1D)

So far, we have considered the energy conservation equation only without material transport (advection). In many cases, however, the material itself moves, and quantities such as temperature, density, etc. must be transported (e.g., in mantle plumes). In general, mantle convection is a good example of a system in which temperature is transported both by **diffusion** (mainly in the boundary layers) and by **advection** (mainly in the interior).  

In the following, we consider **advection** alone (i.e., $k, \kappa = 0$) for a one-dimensional problem.  

In 1D, the energy conservation equation reduces to the pure advection equation:  

$$
\frac{\partial{T}}{\partial{t}} = -v_x \frac{\partial{T}}{\partial{x}}.
$$  

This equation can be solved numerically using different discretization schemes. Here, we will implement several schemes and apply them to a specific advection problem.  

Let us consider two 1-D problems (e.g., a horizontal temperature profile) with specific anomalies:

• A Gaussian temperature profile (a smooth transition):  

<img src="../Figures/Exercise06_gaussian.png" alt="Gaussian temperature profile" width="400" />  

• A block-shaped temperature anomaly (a very sharp transition):  

<img src="../Figures/Exercise06_block.png" alt="Block-shaped temperature anomaly" width="400" />


First, we load the required modules:

In [ ]:
using Plots, Interpolations
using GeoModBox.AdvectionEquation.OneD, GeoModBox.Tracers.OneD
start = time()

We now define the geometry and the necessary numerical parameters (i.e., grid resolution, grid, etc.):

In [ ]:
# Geometric constants --------------------------------------------------- #
xmin    =   0                           # [m]
xmax    =   40                          # [m]
# ----------------------------------------------------------------------- #
# Numerical constants --------------------------------------------------- #
nc      =   100                         # Number of grid points
Δx      =   xmax/nc                     # Grid spacing
# ---
xc      =   xmin + Δx/2 : Δx : xmax - Δx/2   # Cell-centered x-coordinate
xce     =   xmin - Δx/2 : Δx : xmax + Δx/2   # Staggered x-coordinate (with ghost cells)
X       =   zeros(nc)
# ----------------------------------------------------------------------- #
# Maximum simulation time ----------------------------------------------- #
tmax    =   40.0                        # [s]
# ----------------------------------------------------------------------- #
# Horizontal velocity --------------------------------------------------- #
vx      =   1.0                         # [m/s]
# ----------------------------------------------------------------------- #
# Time-step definition -------------------------------------------------- #
Δtfac   =   0.8                         # Courant factor
Δt      =   Δtfac * Δx / abs(vx)        # Time-step size
nt      =   ceil(Int, tmax/Δt)          # Number of time steps
# ----------------------------------------------------------------------- #

We can now choose the finite-difference method (`FTCS`, `upwind`, `downwind`, `lax`, `slf`, `semilag` — the equations and explanations are given in detail in the lecture slides) and select the initial profile (`block` or `gaussian`):

In [ ]:
FD          =   (Method     = (Adv=:upwind,),)
Ini         =   (T=:gaussian,)

For visualization and saving the animation file, the storage location must be specified:

In [ ]:
# Animationssettings ---------------------------------------------------- #
path        =   string("./Results/")
anim        =   Plots.Animation(path, String[] )
filename    =   string("06_1D_advection_",Ini.T,"_",FD.Method.Adv)
save_fig    =   1
# ----------------------------------------------------------------------- #

If we want to use the tracer method, we also need to define the number of tracers per grid spacing:

In [ ]:
# Tracer advection method ----------------------------------------------- #
nmx         =   3       #   Number of tracers per "cell"
# ----------------------------------------------------------------------- #

We now define and plot the initial condition (i.e., the initial temperature profile):

In [ ]:
if Ini.T == :block 
    # Background temperature -------------------------------------------- #
    Tb      =   1000                # [K]
    
    # Location and intensity of the temperature anomaly ----------------- #
    xTl     =   (xmax - xmin) / 10
    xTr     =   xTl + (xmax - xmin) / 10
    Ta      =   1500                # [K]
    
    # Construct initial temperature profile ----------------------------- #
    T       =   Tb .* ones(nc)        
    T[xc .>= xTl .&& xc .<= xTr]    .=  Ta
    Tmin    =   minimum(T)
    Tmax    =   maximum(T)
    tc      =   100
elseif Ini.T == :gaussian
    # Gaussian temperature distribution -------------------------------- #
    Tb      =   1000                # Background temperature
    Ampl    =   500                 # Amplitude
    sigma   =   1                   # Standard deviation
    xcG     =   (xmax - xmin) / 10  # x-coordinate of the maximum
    T       =   zeros(nc)
    @. T    =   Tb + Ampl * exp(-((xc - xcG)^2) / sigma^2)
    
    Tmin    =   minimum(T)
    Tmax    =   maximum(T)
    tc      =   Ampl
end

TWE             =   zeros(nc + 2)
TWE[2:end-1]    .=  T
TWE[1]          =   T[end]
TWE[end]        =   T[1]
TWE2            =   zeros(nc + 2)

q = plot(xc, T, xlabel="x [m]", ylabel="T [°C]", 
        title="Initial temperature distribution", 
        markershape=:circle, label="",
        xlim=(xmin, xmax), ylim=(Tmin - 10, Tmax + 10))
if save_fig == 0
    display(q)
end

if FD.Method.Adv == :tracers
    # Total number of tracers ------------------------------------------- #
    nm          =   (nc) * nmx
    # Tracer spacing ----------------------------------------------------- #
    Δmx         =   (abs(xmin) + abs(xmax)) / (nm + 1)
    # x-coordinates of tracers ------------------------------------------ #
    xm          =   collect(xmin + Δmx : Δmx : xmax - Δmx) .+ rand(nm) .* 0.5 * Δmx    
    # Tracer temperatures ------------------------------------------------ #
    Tm          =   zeros(nm)    
end

We can now implement the equations to solve the advection equation.

In [ ]:
# Solving the advection equation --------------------------------------- #
for i = 2:nt
    display(string("Time step: ", i))

    # Periodic ghost cells (west/east)
    TWE[2:end-1] .= T
    TWE[1]       =  T[end]
    TWE[end]     =  T[1]
    
    if FD.Method.Adv == :FTCS
        # Forward Time, Centered Space (unstable for pure advection)
        T .= TWE[2:end-1] .- (vx * Δt / (2.0 * Δx)) .* (TWE[3:end] .- TWE[1:end-2])

    elseif FD.Method.Adv == :upwind
        # Upwind (first-order, direction depends on sign of vx)
        if vx > 0
            T .= TWE[2:end-1] .- (vx * Δt / Δx) .* (TWE[2:end-1] .- TWE[1:end-2])
        elseif vx < 0
            T .= TWE[2:end-1] .- (vx * Δt / Δx) .* (TWE[3:end] .- TWE[2:end-1])
        end

    elseif FD.Method.Adv == :downwind
        # Downwind (anti-diffusive, generally unstable)
        T .= TWE[2:end-1] .- (vx * Δt / Δx) .* (TWE[3:end] .- TWE[2:end-1])

    elseif FD.Method.Adv == :lax
        # Lax (Lax–Friedrichs) scheme
        T .= (TWE[3:end] .+ TWE[1:end-2]) ./ 2 .-
             (vx * Δt / (2 * Δx)) .* (TWE[3:end] .- TWE[1:end-2])

    elseif FD.Method.Adv == :slf
        # Leapfrog (staggered in time); first step via upwind-like start
        if i == 2
            T .= TWE[2:end-1] .- (vx * Δt / Δx) .* (TWE[3:end] .- TWE[1:end-2])
        else
            T .= TWE2[2:end-1] .- (vx * Δt / Δx) .* (TWE[3:end] .- TWE[1:end-2])
        end
        TWE2 .= TWE

    elseif FD.Method.Adv == :semilag
        # Semi-Lagrangian (cubic spline interpolation)
        X          .= xc .- Δt * vx
        itp_cubic   = cubic_spline_interpolation(xce, TWE)
        T          .= itp_cubic.(X)

    elseif FD.Method.Adv == :tracers
        # Tracer method (NOTE: can be optimized)
        Itp1D_Centers2Markers!(Tm, xm, TWE, xce, Δx, xmin - Δx)
        RK4O1D!(xm, Δt, vx, xmin, xmax)
        Itp1D_Markers2Centers!(T, xc, Tm, xm, Δx, xmin)
    end
    
    display(string("ΔT=", ((Tmax - maximum(T)) / Tmax) * 100))

    # Plot current profile
    q = plot(xc, T, xlabel="x [m]", ylabel="T [°C]",
             title="Temperature profile",
             markershape=:circle, label="",
             xlim=(xmin, xmax), ylim=(Tmin - 10, Tmax + 10))
    
    if FD.Method.Adv == :tracers 
        plot!(xm, Tm, markershape=:circle, label="", linealpha=:0)
    end

    if save_fig == 1
        Plots.frame(anim)
    else
        display(q)
    end
end

# Save animation -------------------------------------------------------- #
if save_fig == 1
    # Write the frames to a GIF file
    Plots.gif(anim, string(path, filename, ".gif"), fps = 15)
    foreach(rm, filter(startswith(string(path, "00")), readdir(path, join=true)))
end
# ----------------------------------------------------------------------- #
stop = time()
println(stop - start)
